

> # Dependencies



In [1]:
!pip install kaggle

In [2]:
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt
import glob
import numpy as np
from tqdm import tqdm
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [3]:
from google.colab import files

# Upload kaggle.json
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shahdhailam","key":"0f461124bf32f4b673c9bfe84b442ac8"}'}

In [4]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d mohammednomer/semantic-segmentation

 95% 65.0M/68.7M [00:00<00:00, 89.5MB/s]
100% 68.7M/68.7M [00:00<00:00, 74.9MB/s]


In [7]:

!mkdir /content/soccer-semantic-segmentation

%cd /content/soccer-semantic-segmentation

zip_path = '/content/semantic-segmentation.zip'

!unzip -q "$zip_path"


/content/soccer-semantic-segmentation


In [8]:
print( len([f for f in os.listdir('/content/soccer-semantic-segmentation') if os.path.isfile(os.path.join('/content/soccer-semantic-segmentation', f))]))

303


> # Dataset preparations

In [9]:
dataset_folder = '/content/soccer-semantic-segmentation'
# list of files in the dataset folder
files = os.listdir(dataset_folder)


# Creating folders and copy files based on their names
for file_name in files:
  if '___' in file_name:
      category = file_name.split('___')[1]
      new_file_name = file_name.split('.')[0]+ '-' + category.split('.')[0] + '.jpg'
      new_file_name = new_file_name.replace(" ", "").replace('Frame1','frame')
      folder_name = 'Frame-' + category.split('.')[0]
  elif 'Frame 1  ' in file_name and '___' not in file_name:
      category = file_name.split()[0]
      new_file_name = file_name.replace(" ", "").replace('Frame1','frame')
      folder_name = category
  else:
    pass


  #Creating a folder for the category if it doesn't exist

  category_folder = os.path.join('/content/Dataset/', folder_name)
  os.makedirs(category_folder, exist_ok=True)

  # Copying the file to the corresponding category folder
  file_path = os.path.join(dataset_folder, file_name)
  destination_path = os.path.join(category_folder, new_file_name)



  #shutil.copy(file_path, destination_path)
  if not os.path.isdir(file_path):
    try:
      image = Image.open(file_path)

      # Resize the image
      image = image.convert('RGB')
      resized_image = image.resize((256, 256))
      resized_image.save(destination_path)
    except:
      print('Not Image')



Not Image
Not Image
Not Image


In [10]:
image_files, images, masks = [], [], []
imgs = os.listdir('/content/Dataset/Frame/')

for i in imgs :

    # Load RGB image
    if '.jpg' in i:
      image_path = os.path.join('/content/Dataset/Frame/', i)
      img = load_img(image_path, target_size=(224, 224))
      img_array = img_to_array(img) / 255.0  # Normalize pixels
      images.append(img_array)

      # Load corresponding mask
      mask_path = os.path.join('/content/Dataset/Frame-fuse/', f"{i.split('.')[0]}-fuse.jpg")
      mask = load_img(mask_path, target_size=(224, 224), color_mode='grayscale')
      mask_array = img_to_array(mask) / 255.0  # Normalize pixels
      masks.append(mask_array)
      image_files.append(i)

image_files

['frame(78).jpg',
 'frame(97).jpg',
 'frame(44).jpg',
 'frame(82).jpg',
 'frame(64).jpg',
 'frame(49).jpg',
 'frame(29).jpg',
 'frame(80).jpg',
 'frame(56).jpg',
 'frame(87).jpg',
 'frame(32).jpg',
 'frame(98).jpg',
 'frame(15).jpg',
 'frame(66).jpg',
 'frame(39).jpg',
 'frame(99).jpg',
 'frame(75).jpg',
 'frame(19).jpg',
 'frame(54).jpg',
 'frame(91).jpg',
 'frame(67).jpg',
 'frame(9).jpg',
 'frame(68).jpg',
 'frame(33).jpg',
 'frame(14).jpg',
 'frame(85).jpg',
 'frame(38).jpg',
 'frame(45).jpg',
 'frame(36).jpg',
 'frame(3).jpg',
 'frame(72).jpg',
 'frame(8).jpg',
 'frame(88).jpg',
 'frame(90).jpg',
 'frame(22).jpg',
 'frame(7).jpg',
 'frame(58).jpg',
 'frame(69).jpg',
 'frame(28).jpg',
 'frame(55).jpg',
 'frame(89).jpg',
 'frame(81).jpg',
 'frame(70).jpg',
 'frame(20).jpg',
 'frame(52).jpg',
 'frame(93).jpg',
 'frame(6).jpg',
 'frame(4).jpg',
 'frame(18).jpg',
 'frame(23).jpg',
 'frame(96).jpg',
 'frame(94).jpg',
 'frame(5).jpg',
 'frame(11).jpg',
 'frame(77).jpg',
 'frame(60).jpg',

In [11]:
images = np.array(images)
masks = np.array(masks)

In [12]:
print('Fuse Folder : ', len([f for f in os.listdir('/content/Dataset/Frame-fuse') if os.path.isfile(os.path.join('/content/Dataset/Frame-fuse', f))]))
print('Save Folder : ', len([f for f in os.listdir('/content/Dataset/Frame-save') if os.path.isfile(os.path.join('/content/Dataset/Frame-save', f))]))
print('Frame Folder : ', len([f for f in os.listdir('/content/Dataset/Frame') if os.path.isfile(os.path.join('/content/Dataset/Frame', f))]))

Fuse Folder :  100
Save Folder :  100
Frame Folder :  100


> # U Net


In [13]:
train_images, val_images, train_masks, val_masks = train_test_split(
    images, masks, test_size=0.2, random_state=42, shuffle=True
)

In [14]:
train_masks = to_categorical(train_masks, num_classes=11)
val_masks = to_categorical(val_masks, num_classes=11)

In [15]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [16]:
model = Sequential(name='U-Net')

# Encoder
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Bottleneck
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))

# Decoder
model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))

model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

model.add(UpSampling2D(size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

# Output
model.add(Conv2D(11, (1, 1), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "U-Net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                        

> # Final Output

In [17]:
history = model.fit(
    datagen.flow(train_images, train_masks, batch_size=32),
    epochs=10,
    validation_data=(val_images, val_masks)
)

Epoch 1/10
3/3 [==============================] - 75s 23s/step - loss: 1.2844 - accuracy: 0.8001 - val_loss: 1.5341e-09 - val_accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 3s 758ms/step - loss: 7.2112e-08 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 3s 987ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 3s 1s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 3s 738ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 3s 685ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 3s 680ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000

In [18]:
test_loss, test_accuracy = model.evaluate(val_images, val_masks)

1/1 [==============================] - 0s 358ms/step - loss: 0.0000e+00 - accuracy: 1.0000
